# The Freva command line interface

This notebook introduces the usage of the Freva command line interface - cli. The tutorial assumes that you have already access to Freva either because you’ve setup an instance yourself or one has been setup by the Freva admin team. Hence it is assumed that you know how to access Freva. If this is not the case please contact one of your Freva admins for help.

A general usage overview of the available Freva sub-commands is available via the --help option:

In [4]:
freva --help

usage: freva [-h] [-V] {databrowser,plugin,history,user-data,esgf} ...

Free EVAluation system framework (freva)

positional arguments:
  {databrowser,plugin,history,user-data,esgf}
                        Available sub-commands:
    databrowser         Find data in the system.
    plugin              Apply data analysis plugin.
    history             Read the plugin application history.
    user-data           Update users project data
    esgf                Search/Download ESGF the data catalogue.

optional arguments:
  -h, --help            show this help message and exit
  -V, --version         show program's version number and exit

To get help for the individual sub-commands use: freva <sub-command> --help


The most common sub-commands are `databrowser` and `plugin`. You can get more help on the actual commands using the sub-commands `--help` option, for example getting help on the databrowser usage:

In [6]:
freva databrowser --help

usage: freva databrowser [-h] [--multiversion] [--relevant-only]
                         [--batch-size BATCH_SIZE] [--count] [--attributes]
                         [--all-facets] [--facet FACET]
                         [--facet-limit FACET_LIMIT]
                         [--time-select {flexible,strict,file}] [--debug]
                         [facets ...]

Find data in the system.

positional arguments:
  facets                Search facet(s) (default: None)

optional arguments:
  -h, --help            show this help message and exit
  --multiversion        Select not only the latest version. (default: False)
  --relevant-only       Show only search with results >1 possible values
                        (default: False)
  --batch-size BATCH_SIZE
                        Number of files to retrieve (default: 5000)
  --count               Show the number of files for each search result
                        (default: False)
  --attributes          Retrieve all possible attributes f

__Note__: Instead of using sub-commands you have also the option to use commands. For example the command `freva-databrowser` is equivalent to `freva databrowser`, `freva-user-data` to `freva user-data` etc.

## Searching for data: the freva-databrowser command

All files available on in the project are scanned and indexed via a data search server. This allows you to query the server which responds almost immediately. To search for data you can either use the `freva-databrowser` command or the `freva databrowser` sub-command. Let’s inspect the help menu of the databrowser sub-command:

In [7]:
freva-databrowser --help

usage: freva-databrowser [-h] [--multiversion] [--relevant-only]
                         [--batch-size BATCH_SIZE] [--count] [--attributes]
                         [--all-facets] [--facet FACET]
                         [--facet-limit FACET_LIMIT]
                         [--time-select {flexible,strict,file}] [--debug] [-V]
                         [facets ...]

Find data in the system.

positional arguments:
  facets                Search facet(s) (default: None)

optional arguments:
  -h, --help            show this help message and exit
  --multiversion        Select not only the latest version. (default: False)
  --relevant-only       Show only search with results >1 possible values
                        (default: False)
  --batch-size BATCH_SIZE
                        Number of files to retrieve (default: 5000)
  --count               Show the number of files for each search result
                        (default: False)
  --attributes          Retrieve all possible attribu

The databrowser expects a list of `key=value` pairs. The order of the pairs doesn’t really matter. Most important is that you don’t need to split the search according to the type of data you are searching for. You can search for any files, for example observations, reanalysis and model data at the same time. Also important is that all queries are case insensitive. You can also search for attributes themselves instead of file paths. For example you can search for the list of variables available that satisfies a certain constraint (e.g. sampled 6hr, from a certain model, etc).

In [8]:
freva-databrowser project=observations variable=pr model=cp*

/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022300-201609022330.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022200-201609022230.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022100-201609022130.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022000-201609022030.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609021900-201609021930.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30

In some cases it might be useful to know how many files are found in the `databrowser` for certain search constraints. In such cases you can use the `count` flag to count the number of found files instead of getting the files themselves.

In [9]:
freva-databrowser project=observations --count

24


Sometimes it might be useful to subset the data you’re interested in by time. To do so you can use the time search key to subset time steps and whole time ranges. For example let’s get the for certain time range:

In [11]:
freva-databrowser project=observations time='2016-09-02T22:15 to 2016-10'

/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022300-201609022330.nc
/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022200-201609022230.nc


The default method for selecting time periods is `flexible`, which means all files are selected that cover at least start or end date. The `strict` method implies that the entire search time period has to be covered by the files. Using the `strict` method in the example above would only yield on file because the first file contains time steps prior to the start of the time period:

In [12]:
freva-databrowser project=observations time='2016-09-02T22:15 to 2016-10' --time-select strict

/home/wilfred/workspace/evaluation_system/.docker/data/observations/grid/CPC/CPC/cmorph/30min/atmos/30min/r1i1p1/v20210618/pr/pr_30min_CPC_cmorph_r1i1p1_201609022300-201609022330.nc


You might as well want to know about possible values that an attribute can take after a certain search is done. For this you use the `--facet` flag (facets are the attributes used to search for and sub set the data). For example to see all `facets` that are available in the observations project:

In [13]:
freva-databrowser project=observations --all-facets

product: grid
project: observations
time_frequency: 30min
institute: cpc
dataset: observations
model: cpc
variable: pr
cmor_table: 30min
experiment: cmorph
ensemble: r1i1p1
realm: atmos


In [14]:
freva-databrowser --facet time_frequency --facet variable project=observations

time_frequency: 30min
variable: pr


You can also retrieve information on how many facets are found by the databrowser by giving the `count` flag

In [16]:
freva-databrowser --facet time_frequency --facet variable project=observations --count

time_frequency: 30min (24)
variable: pr (24)


# Running data analysis plugins: the freva-plugin command

Already defined data analysis tools can be started with the `freva-plugin` command or the `freva plugin` sub-command. Let’s inspect the help menu of the plugin command:

In [17]:
freva-plugin --help

usage: freva-plugin [-h] [--repo-version] [--caption CAPTION] [--save]
                    [--save-config SAVE_CONFIG] [--show-config] [--dry-run]
                    [--batchmode] [--unique_output UNIQUE_OUTPUT]
                    [--pull-request] [--tag TAG] [--debug] [--list-tools]
                    [--doc] [-V]
                    [plugin_name]

Apply data analysis plugin.

positional arguments:
  plugin_name           Plugin name (default: None)

optional arguments:
  -h, --help            show this help message and exit
  --repo-version        Show the version number from the repository (default:
                        False)
  --caption CAPTION     Set a caption for the results (default: )
  --save                Save the plugin configuration to default destination.
                        (default: False)
  --save-config SAVE_CONFIG
                        Save the plugin configuration. (default: None)
  --show-config         Show the resulting configuration (implies dry-ru

As the help menu suggests you can list all available tools using the `-l` option (or `--list`, `--list-tools`):

In [18]:
freva-plugin -l

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DummyPlugin: A dummy plugin


This means currently we have two plugins available (animator and dummyplugin). The general syntax is freva-plugin `<plugin-name> [options]` for example to inspect the documentation of a certain plugin you can use the `--doc` option. Here we concentrate on the Animator plugin. A simple plugin that creates animations of geospatial data. The basic usage of that command can be retrieved by:

In [19]:
freva-plugin --doc animator

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
evaluation_system.misc.exceptions.PluginNotFoundError: 
animator plugin not found, did you mean:



: 1

The parameters are also given as `key=values` pairs. But not all of the above parameters are mandatory. Let’s use one project search key and animate its content.

In [20]:
freva plugin animator project=observations variable=pr cmap=Blues fps=5 output_unit=mm/h vmin=0 vmax=5 suffix=gif

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
evaluation_system.misc.exceptions.PluginNotFoundError: 
animator plugin not found, did you mean:



: 1

This plugin will run in so called interactive mode. That means that it will run on the login node and block your shell until the command is completed. This can be problematic if you have jobs that might take time to finish. An alternative is setting the `-–batchmode` flag. This flag tells the plugin to submit a job to the computing queue. The computing nodes are the core of any high performance computing system. Let’s submit the previous plugin job to the computing queue:

In [21]:
freva plugin animator project=observations variable=pr cmap=Blues fps=5 output_unit=mm/h vmin=0 vmax=5 suffix=gif --batchmode

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
evaluation_system.misc.exceptions.PluginNotFoundError: 
animator plugin not found, did you mean:



: 1

# Inspecting previous analysis jobs: the freva-history command

Sometimes it can be useful to access the status of past plugin applications. The `freva-history` command or `freva history` sub-command can do that:

In [25]:
freva-history --limit 1 --plugin dummyplugin

INFO:freva:history, limit=1, since=None, until=None, entry_ids=None
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
2111) animator [2022-09-29 19:04:29] broken {"input_file": null, "variable": "pr", "project": "obs*", "product"...


In [24]:
freva-history --entry-ids 1 --full-text

INFO:freva:history, limit=10, since=None, until=None, entry_ids=['1']
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


To re-run a command of a past configuration you can use the --return-command option to get the command that was used:

In [26]:
freva-history  --entry-ids 136 --return-command

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# Managing your own datasets: the freva-user-data command

Freva offers the possibility to share custom datasets with other users by making it searchable via `freva-databrowser`. With help of the `freva-user-data` command users can add their own data to the central data location, (re)-index or delete data in the databrowser.

__Note__: Any data that has been added by users will be assigned a special project name: `project=user-$USER`.
Let’s inspect the help menu of the freva-user-data or freva user-data command:

In [27]:
freva-user-data --help

usage: freva-user-data [-h] [-V] {index,add,delete} ...

Update users project data

positional arguments:
  {index,add,delete}  Available sub-commands:
    index             Index existing user project data to the databrowser
    add               Add new user project data to the databrowser
    delete            Delete existing user project data from the databrowser

optional arguments:
  -h, --help          show this help message and exit
  -V, --version       show program's version number and exit


## Add new data to the databrowser

To be able to add data to the databrowser the file names must follow a strict standard and the files must reside in a specific location. The add sub command takes care about the correct file naming and location. No pre requirements other than the file has to be a valid netCDF or grib file are assumed. In other words this sub command places the user data with the correct naming structure to the correct location.

In [28]:
freva-user-data add --help

usage: freva-user-data add [-h] [--how {copy,move,symlink,link}] [--override]
                           [--experiment EXPERIMENT] [--institute INSTITUTE]
                           [--model MODEL] [--variable VARIABLE]
                           [--time_frequency TIME_FREQUENCY]
                           [--ensemble ENSEMBLE] [--debug]
                           product paths [paths ...]

Add new user project data to the databrowser

positional arguments:
  product               Product search key the newly added data can be found.
  paths                 Filename(s) or Directories that are going to be added
                        to thedatabrowser

optional arguments:
  -h, --help            show this help message and exit
  --how {copy,move,symlink,link}
                        Method of how the data is added into the central freva
                        user directory. (default: copy)
  --override, --overwrite
                        Replace existing files in the user data struc

Suppose you’ve gotten data from somewhere and want to add this data into the databrowser to make it accessible to others. In this specific example we assume that you have stored your original data in the `/tmp/my_awesome_data` folder. E.g `/tmp/my_awesome_data/outfile_0.nc...tmp/my_awesome_data/outfile_9.nc` The routine will try to gather all necessary metadata from the files. You’ll have to provide additional metadata if mandatory keywords are missing. To make the routine work in this example we have to provide the institute, model and experiment keywords:



In [29]:
freva-user-data add eur-11b /tmp/my_awesome_data/outfile_?.nc \
--institute clex --model UM-RA2T --experiment Bias-correct
freva-databrowser experiment=bias-correct

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Status: crawling ...ok
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-RA2T_Bias-correct_r0i0p0_197001041800-197001050300.nc
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-RA2T_Bias-correct_r0i0p0_197001040800-197001041700.nc
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-RA2T_Bias-correct_r0i0p0_197001032200-197001040700.nc
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-RA2T_Bias-correct_r0i0p0_197001031200-197001032100.nc
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-RA2T_Bias-correct_r0i0p0_197001030200-197001031100.nc
/tmp/user_data/user-wilfred/eur-11b/clex/UM-RA2T/Bias-correct/hr/user_data/hr/r0i0p0/v20220930/tas/tas_hr_UM-R

## Remove your data from the databrowser

The delete sub command removes entries from the databrowser and if necessary existing files from the central user data location.

In [30]:
freva-user-data delete --help

usage: freva-user-data delete [-h] [--delete-from-fs] [--debug]
                              paths [paths ...]

Delete existing user project data from the databrowser

positional arguments:
  paths                 The user directory(s) that needs to be crawled

optional arguments:
  -h, --help            show this help message and exit
  --delete-from-fs, --delete_from_fs, --delete
                        Do not only delete the files from the databrowser but
                        also from file system. (default: False)
  --debug, -v, -d, --verbose
                        Use verbose output. (default: False)


Any data in the central user directory that belongs to the user can be deleted from the databrowser and also from the central data location:

In [31]:
freva-user-data delete /tmp/user_data/user-$USER
freva-databrowser experiment=bias-correct

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## (Re)-Index existing data to the databrowser

The index subcommand can be used to update the databrowser for existing user data. For example, if data has been removed from the databrowser it can be re-added:

In [32]:
freva-user-data index --help

usage: freva-user-data index [-h] [--data-type {fs}] [--debug] [crawl_dir ...]

Index existing user project data to the databrowser

positional arguments:
  crawl_dir             The user directory(s) that needs to be crawled
                        (default: None)

optional arguments:
  -h, --help            show this help message and exit
  --data-type {fs}, --dtype {fs}
                        The data type of the data. (default: fs)
  --debug, -v, -d, --verbose
                        Use verbose output. (default: False)


In [33]:
freva-user-data delete /tmp/user_data/user-$USER
freva-databrowser experiment=bias-correct


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
